# Setup

In [12]:
import pandas as pd
import numpy as np
import nba_api
import json
import time

In [13]:
season = '2019-20'

## nba_api packages

In [14]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.static.players import get_active_players, find_players_by_full_name
from nba_api.stats.endpoints import CommonTeamRoster

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import LeagueDashTeamShotLocations
from nba_api.stats.endpoints import ShotChartLineupDetail
from nba_api.stats.endpoints import ShotChartDetail
from nba_api.stats.endpoints import BoxScorePlayerTrackV2

## Get spurs test game

In [15]:
# Get Spurs' team info
all_teams = get_teams()
spurs_info = find_teams_by_nickname('spurs') # find spurs nickname
spurs_id = spurs_info[0]['id'] # get spurs team ID

# find all Spurs games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
spurs_games_df = gamefinder.get_data_frames()[0]

# test - last 2019-20 regular season game vs Utah Jazz
test_game_id = spurs_games_df.iloc[0]['GAME_ID']

spurs_games_df.head(n=5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612759,SAS,San Antonio Spurs,0021901314,2020-08-13,SAS @ UTA,L,239,112,...,0.667,9,35,44,24,4,8,15,20,-6.0
1,22019,1610612759,SAS,San Antonio Spurs,0021901298,2020-08-11,SAS vs. HOU,W,240,123,...,0.900,9,50,59,28,10,5,22,20,18.0
2,22019,1610612759,SAS,San Antonio Spurs,0021901287,2020-08-09,SAS @ NOP,W,241,122,...,0.969,15,36,51,22,12,6,17,27,9.0
3,22019,1610612759,SAS,San Antonio Spurs,0021901274,2020-08-07,SAS vs. UTA,W,240,119,...,0.769,8,39,47,27,10,5,13,18,8.0
4,22019,1610612759,SAS,San Antonio Spurs,0021901264,2020-08-05,SAS vs. DEN,L,239,126,...,0.852,10,25,35,29,10,5,7,20,-6.0


# Functions

In [16]:
def get_playerid(name):
    return find_players_by_full_name(name)[0]['id']

In [20]:
def get_shot_chart(playerid, teamid, gameid):
    """
    This function takes in a player and returns his shot chart for a specific game.
    
    Inputs:
    playerid: Player's unique ID
    teamid: Associated team ID
    gameid: Specific game ID
    
    Output: Entire DataFrame with shots data
    """
    shots_df = ShotChartDetail(
        player_id = playerid,
        season_nullable = season,
        team_id = teamid,
        game_id_nullable = gameid,
        context_measure_simple = 'FGA'
    )
    
    return shots_df.data_sets[0].get_data_frame()

In [24]:
def gather_team_df(spurs_or_opp):
    """
    This function grabs an entire team's shot data separated by each shot. 
    It loops through a team's roster and grabs individual team's shot data.
    
    Input: 
    spurs_or_opp: True or False
    
    Output: concatted DataFrame of a team's game shot data
    """
    if spurs_or_opp == True:
        player_list = spurs_roster_list
        team_id = spurs_id
    else:
        player_list = opp_roster_list
        team_id = opp_teamid
        
    data = pd.DataFrame() # start with an empty dataframe
    
    for player in player_list:
        df = get_shot_chart(player_ids[player], team_id, test_game_id)
        data = pd.concat([data, df])
        # put a sleep timer of a few seconds
        
    return data

# Get Spurs and opponent rosters

In [17]:
roster_data_df = BoxScorePlayerTrackV2(game_id=test_game_id).data_sets[0].get_data_frame()
spurs_roster_list = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_roster_list = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)

In [18]:
# get opponent team ID
opp_teamid = roster_data_df[roster_data_df.TEAM_ABBREVIATION != 'SAS'].TEAM_ID.iloc[0]

## Create dictionary of Player:ID 

In [19]:
player_ids = dict()

for player in spurs_roster_list:
    player_ids[player] = get_playerid(player)
    
for player in opp_roster_list:
    player_ids[player] = get_playerid(player)

# Test package outputs

In [25]:
testdf = gather_team_df(True)

In [27]:
testdf.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [33]:
testdf.SHOT_ZONE_RANGE.unique()

array(['Less Than 8 ft.', '8-16 ft.', '24+ ft.', '16-24 ft.'],
      dtype=object)

In [28]:
testdf.SHOT_ZONE_AREA.unique()

array(['Center(C)', 'Right Side(R)', 'Left Side Center(LC)',
       'Left Side(L)', 'Right Side Center(RC)'], dtype=object)

In [22]:
# get rosters of each team
# grab separate shot charts for a game
# create a file for each team
# separate by sections 

In [51]:
# group dataframe by team, shot zone area, shot zone range
# sum FGM & FGA 

agg_testdf = testdf.groupby(['GAME_ID', 'TEAM_NAME', 'SHOT_ZONE_AREA',
                            'SHOT_ZONE_RANGE'])[
    'GAME_ID', 'TEAM_NAME', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_MADE_FLAG',
    'SHOT_ATTEMPTED_FLAG'
].sum().reset_index()

<ipython-input-51-d8b5ef5ff70f>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_testdf = testdf.groupby(['GAME_ID', 'TEAM_NAME', 'SHOT_ZONE_AREA',


In [56]:
agg_testdf['fg_pct'] = round((agg_testdf.SHOT_MADE_FLAG / agg_testdf.SHOT_ATTEMPTED_FLAG), 2) * 100
agg_testdf

,GAME_ID,TEAM_NAME,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_MADE_FLAG,SHOT_ATTEMPTED_FLAG,fg_pct
0,0021901314,San Antonio Spurs,Center(C),24+ ft.,2,2,100.0
1,0021901314,San Antonio Spurs,Center(C),8-16 ft.,3,10,30.0
2,0021901314,San Antonio Spurs,Center(C),Less Than 8 ft.,17,30,57.0
3,0021901314,San Antonio Spurs,Left Side Center(LC),24+ ft.,3,6,50.0
4,0021901314,San Antonio Spurs,Left Side(L),16-24 ft.,3,5,60.0
5,0021901314,San Antonio Spurs,Left Side(L),24+ ft.,1,3,33.0
6,0021901314,San Antonio Spurs,Left Side(L),8-16 ft.,1,1,100.0
7,0021901314,San Antonio Spurs,Right Side Center(RC),16-24 ft.,0,3,0.0
8,0021901314,San Antonio Spurs,Right Side Center(RC),24+ ft.,3,7,43.0
9,0021901314,San Antonio Spurs,Right Side(R),16-24 ft.,1,4,25.0


In [59]:
# rename shot data columns to lower case
shot_cols = dict()

for col in list(agg_testdf.columns):
    shot_cols[col] = col.lower()
    
agg_testdf.rename(columns = shot_cols, inplace=True)

In [60]:
#agg_testdf.to_json('data/test_spurs_shots.json', orient='records')

In [42]:
# get league average shot chart

name = get_playerid(spurs_roster_list[0])

league_avg = ShotChartDetail(
    season_nullable=season,
    team_id=spurs_id,
    player_id=name,
    context_measure_simple='FGA'
)

In [44]:
league_avg_df = league_avg.data_sets[1].get_data_frame() # league averages is 1
league_avg_df

,GRID_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,FGA,FGM,FG_PCT
0,League Averages,Above the Break 3,Back Court(BC),Back Court Shot,45,6,0.133
1,League Averages,Above the Break 3,Center(C),24+ ft.,14955,5168,0.346
2,League Averages,Above the Break 3,Left Side Center(LC),24+ ft.,20895,7459,0.357
3,League Averages,Above the Break 3,Right Side Center(RC),24+ ft.,19624,6875,0.350
4,League Averages,Backcourt,Back Court(BC),Back Court Shot,405,7,0.017
5,League Averages,In The Paint (Non-RA),Center(C),8-16 ft.,9820,4159,0.424
6,League Averages,In The Paint (Non-RA),Center(C),Less Than 8 ft.,16915,6518,0.385
7,League Averages,In The Paint (Non-RA),Left Side(L),8-16 ft.,1770,714,0.403
8,League Averages,In The Paint (Non-RA),Right Side(R),8-16 ft.,1871,738,0.394
9,League Averages,Left Corner 3,Left Side(L),24+ ft.,8380,3329,0.397


In [61]:
#league_avg_df.to_json('data/test_league_avg_shots.json', orient = 'records')